# Data Science Bootcamp - Unit 1 Lesson 3.6
## Data cleaning & validation

- determine the five most common journals
- and the total articles for each

- calculate the mean, median, and standard deviation of the open-access cost per article for each journal


# Imports

In [0]:
import numpy as np
import pandas as pd
import re



# Import CSV file
encoding mac_roman must be used to decode the Pound Sterling symbol 

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/Vonganization/CSV_datasets/master/WELLCOME_APCspend2013_forThinkful.csv', encoding='mac_roman')
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


# Understanding the data
To see if there are any null values and if the dataset needs cleaning

In [0]:
df.shape

(2127, 5)

In [0]:
df.info()
# I can see that PMID/PMCID, and Journal Title has null values...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


## Taking a look at all the null/NaN values within the PMID column

In [0]:
# Bringing up all the values in the PMID/PMCID column that have null values
df[df['Journal title'].isnull()]

# PMID/PMCID shouldn't matter to us because we are interested in Journals and Articles 
# Looking at this one record, I don't believe it will effect our results

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
986,NaN,MacMillan,NaN,Fungal Disease in Britain and the United State...,£13200.00


In [0]:
df['Journal title'].groupby(df['Journal title']).agg('count')
# We can see that upper and lowercase, spacing, and special characters are causing problems

Journal title
ACS Chemical Biology                                                                       5
ACS Chemical Neuroscience                                                                  1
ACS NANO                                                                                   1
ACS Nano                                                                                   1
ACTA F                                                                                     1
AGE                                                                                        1
AIDS                                                                                       3
AIDS Behav                                                                                 1
AIDS Care                                                                                  2
AIDS Journal                                                                               1
AIDS Research and Therapy                               

# Cleaning Data
- remove all spaces
- remove special characters (, : &)
- make all text lowercase
- change number "1" to "one"

In [0]:
df_clean_jt = df['Journal title'].str.replace(' ','').str.replace(',','').str.replace(':','').str.replace('&','and').str.lower().str.replace('1','one')
df_clean_jt


0                                  psychologicalmedicine
1                                      biomacromolecules
2                                               jmedchem
3                                               jmedchem
4                                               jorgchem
5                            journalofmedicinalchemistry
6                              journalofproteomeresearch
7                                               molpharm
8                                     acschemicalbiology
9                                     acschemicalbiology
10               journalofchemicalinformationandmodeling
11                           journalofmedicinalchemistry
12                           journalofmedicinalchemistry
13                                          biochemistry
14                           journalofmedicinalchemistry
15                                      gastroenterology
16                          journalofbiologicalchemistry
17                             

## Determine the five most common journals, and total articles for each

In [0]:
top_5_jt = df_clean_jt.groupby(df_clean_jt).agg('count').reset_index(name='count')
top_5_jt.sort_values('count', ascending=False).head(5)

,Journal title,count
732,plosone,207
490,journalofbiologicalchemistry,53
667,neuroimage,29
689,nucleicacidsresearch,25
726,plosgenetics,24


## Clean data
- Remove Sterling Pound symbol
- Remove 999999.00, as it does not appear to have any particular value

In [0]:
df_cost = df['COST (£) charged to Wellcome (inc VAT when charged)'].str.slice(start=1).str.replace('999999.00','').reset_index(name='money')

## Change type of data in column to numeric so that I can apply statistics

In [0]:
df_cost_num = pd.to_numeric(df_cost['money'], errors='coerce')

## Calculate the mean, median, and standard deviation of the open-access cost per article for each journal

In [0]:
df_cost_num.describe()

count      2067.000000
mean       2018.609173
std        6119.374144
min           0.000000
25%        1268.700000
50%        1863.000000
75%        2303.735000
max      201024.000000
Name: money, dtype: float64

# Afterthoughts

After going back and reviewing the data, I can see that more cleaning is needed... for instance, there are journal titles with an "s" to make the word plural, whereas other titles are not plural